In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI # Import the Gemini model
from dotenv import load_dotenv
load_dotenv()

/home/mukundan/projects/llm_app/supervisor_doctor_agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully connected to Milvus.
Collection face_collection already exists.  Loading...
Connected to PostgreSQL
Tables created successfully.


True

In [2]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from agents.specialized_agent import get_context

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Create specialized agents
dermatology_tool = get_context("dermatology")
gastroentology_tool = get_context("gastroentology")

# def add(a: float, b: float) -> float:
#     """Add two numbers."""
#     return a + b

# def multiply(a: float, b: float) -> float:
#     """Multiply two numbers."""
#     return a * b

# def web_search(query: str) -> str:
#     """Search the web for information."""
#     return (
#         "Here are the headcounts for each of the FAANG companies in 2024:\n"
#         "1. **Facebook (Meta)**: 67,317 employees.\n"
#         "2. **Apple**: 164,000 employees.\n"
#         "3. **Amazon**: 1,551,000 employees.\n"
#         "4. **Netflix**: 14,000 employees.\n"
#         "5. **Google (Alphabet)**: 181,269 employees."
#     )

dermatology_agent = create_react_agent(
    model=model,
    tools=[dermatology_tool],
    name="dermatologist",
    prompt="You are a dermatological expert.Use the RAG to query the textbook and propely ground with facts. Always use one tool at a time."
)

gastroentology_agent = create_react_agent(
    model=model,
    tools=[gastroentology_tool],
    name="gastroentologist",
    prompt="You are a gastroentological expert.Use the RAG to query the textbook and propely ground with facts. Always use one tool at a time."
)

# Create supervisor workflow
workflow = create_supervisor(
    [gastroentology_agent, dermatology_agent],
    model=model,
    prompt=(
        "You are a general physician handling along with a team consisting  a dermatology_agent and a gastroentology_agent "
        "For gastric issues, use gastroentology_agnet. "
        "For skin issues, use dermatology_agent."
        "If it is pertaining other domains give ypu own general answer "
    )
)

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Patient presents with a short bowel syndrome "
        }
    ]
})

<Collection>:
-------------
<name>: gastroentology_rag_collection
<description>: RAG data for gastroentology
<schema>: {'auto_id': True, 'description': 'RAG data for gastroentology', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}], 'enable_dynamic_field': False}
 collection found 


/home/mukundan/projects/llm_app/supervisor_doctor_agent/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Results ["id: 456303077940532196, distance: 0.8231000900268555, entity: {'text': '265\\nYamada’s Handbook of Gastroenterology, Fourth Edition. John M. Inadomi,  \\nRenuka Bhattacharya, Joo Ha Hwang, and Cynthia Ko. \\n© 2020 John Wiley & Sons Ltd. Published 2020 by John Wiley & Sons Ltd.\\nCHAPTER\\xa024\\nShort bowel syndrome is a condition resulting from surgical resection, congen-\\nital defect or disease‐associated loss of absorption from a substantial portion of \\nsmall or large intestine. Less than 200 cm of remaining small intestine is often'}", "id: 456303077940541916, distance: 0.8230999112129211, entity: {'text': '265\\nYamada’s Handbook of Gastroenterology, Fourth Edition. John M. Inadomi,  \\nRenuka Bhattacharya, Joo Ha Hwang, and Cynthia Ko. \\n© 2020 John Wiley & Sons Ltd. Published 2020 by John Wiley & Sons Ltd.\\nCHAPTER\\xa024\\nShort bowel syndrome is a condition resulting from surgical resection, congen-\\nital defect or disease‐associated loss of absorption from a 

In [3]:
for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

Patient presents with a short bowel syndrome 
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_gastroentologist (4528036b-9810-4901-ab66-5e1dd1e5a9e4)
 Call ID: 4528036b-9810-4901-ab66-5e1dd1e5a9e4
  Args:
================================= Tool Message =================================
Name: transfer_to_gastroentologist

Successfully transferred to gastroentologist
================================== Ai Message ==================================
Name: gastroentologist

Short bowel syndrome (SBS) is a malabsorptive condition that typically arises after extensive resection of the small intestine. The clinical diagnosis of SBS is based on how ever, more meaningful clinical criteria for short bowel associated intestinal failure is absorption <1.4 kg/day of wet weight, and/or need for >84% of energy needs.

Common signs and symptoms incl